In [1]:
!pip install transformers datasets trl huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00


In [3]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import evaluate  # To calculate ROUGE metrics
from datasets import DatasetDict

In [4]:
data_path = 'Recommendation.csv'
df = pd.read_csv(data_path)
dataset = Dataset.from_pandas(df)

In [5]:
# Dataset'in %40'lık subset'ini al
subset_fraction = 0.4
subset_dataset = dataset.train_test_split(test_size=1 - subset_fraction, seed=42)['train']

In [6]:
# Subset'ten DataFrame oluştur
subset_df = subset_dataset.to_pandas()

In [8]:
def process_data_for_dpo(example):
    messages = [{"role": "user", "content": f"Recommend a food list for a person whose gender is {example['Cinsiyet']}, age: {example['Yaş']}, hypertension: {example['Hipertansiyon']}, diabetes: {example['Diyabet']}, give only 5 foods. Respond only the names of recommended foods."},
                {"role": "assistant", "content": ", ".join(eval(example['Recommended Foods']))}]
    return {"messages": messages}

In [9]:

# Eğitim ve doğrulama için split (subset üzerinde)
train_df, val_df = train_test_split(subset_df, test_size=0.2, random_state=42)

In [10]:

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [11]:
# Veri setlerini işlemden geçirme
processed_train_dataset = train_dataset.map(process_data_for_dpo)
processed_val_dataset = val_dataset.map(process_data_for_dpo)

Map:   0%|          | 0/1977 [00:00<?, ? examples/s]

Map:   0%|          | 0/495 [00:00<?, ? examples/s]

In [12]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=640b4ca5a6254777cc450d4d00b8811599d4671274258d8cdd44e7630f70e8b2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# Metric computation
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions[0]

    # Padding tokenlarını düzgün işleme
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id

    # Tahminleri ve referansları decode etme
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # String'leri temizle ve küçük harf standardizasyonu yap
    pred_str = [p.strip().lower().split(", ")[:5] for p in pred_str]  # İlk 5 yemek
    label_str = [l.strip().lower().split(", ")[:5] for l in label_str]  # İlk 5 yemek

    # Accuracy hesaplama: Ground truth yemeklerin kapsanma oranı
    correct_count = 0
    total_count = len(label_str)

    for pred_list, label_list in zip(pred_str, label_str):
        correct_count += len(set(pred_list) & set(label_list)) / len(label_list)

    accuracy = correct_count / total_count if total_count > 0 else 0.0

    return {
        "Accuracy": round(accuracy, 4),
    }

In [ ]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Memory optimization: avoids storing unnecessary tensors.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [12]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

model.config.use_cache = False


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

In [13]:
os.environ["WANDB_DISABLED"] = "true"

# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=200,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=1,  # Set according to your GPU memory capacity
    gradient_accumulation_steps=4,
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=50,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    # eval_steps=50,  # Frequency of evaluation
    fp16=True,  # Mixed precision
)


trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_val_dataset,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
    #preprocess_logits_for_metrics=preprocess_logits_for_metrics,  # Optimize memory usage
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-13-3fdf669ab883>:18: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/1977 [00:00<?, ? examples/s]

Map:   0%|          | 0/495 [00:00<?, ? examples/s]

In [14]:
# Train the model
trainer.train()

# Modeli kaydetme
finetune_name = "SmolLM2-FT-MyDataset"
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
50,0.734200,0.206951
100,0.159500,0.154497
150,0.138600,0.135250
200,0.132700,0.128498


In [15]:
prompt = "Recommend a food list for a person whose gender is male, age: 53, hypertension: yes, diabetes: yes, give only 5 foods. Respond only the names of recommended foods"
test_prompt = [{"content": prompt, "role": "user"}]

formatted_prompt = tokenizer.apply_chat_template(test_prompt, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

In [18]:
import random  # Bu satırı ekleyin

outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=100)
output = tokenizer.decode(outputs[0], skip_special_tokens=True)
# Listeyi böl
food_list = output.split("assistant\n")[1].split(",")

# Rastgele 5 tane seç
food = random.sample(food_list, min(len(food_list), 5))

print(food)

[' Espresso', ' Focaccia', ' Cheesecake', ' Onion rings', ' Custard']


In [20]:
def compute_accuracy(expected, predicted):
    """
    Gerçek (expected) ve tahmin edilen (predicted) listeler arasındaki eleman bazında eşleşme sayısını hesaplar.
    """
    # Elemanları temizle ve küçük harfe çevirerek normalize et
    expected_set = set(map(str.strip, map(str.lower, eval(expected))))
    predicted_list = list(map(str.strip, map(str.lower, predicted)))

    # Eşleşmeleri say
    match_count = sum(1 for item in predicted_list if item in expected_set)

    # Doğruluk oranı: eşleşen eleman sayısı
    return match_count


In [21]:
from tqdm import tqdm

total_matches = 0  # Toplam eşleşme sayısını tutacak değişken
for person in tqdm(processed_val_dataset, desc="Processing dataset"):
    prompt = f"Recommend a food list for a person whose gender is {person['Cinsiyet']}, age: {person['Yaş']}, hypertension: {person['Hipertansiyon']}, diabetes: {person['Diyabet']}, give only 5 foods. Respond only the names of recommended foods"
    test_prompt = [{"content": prompt, "role": "user"}]

    formatted_prompt = tokenizer.apply_chat_template(test_prompt, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

    outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=100)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Listeyi böl
    food_list = output.split("assistant\n")[1].split(",")

    # Rastgele 5 tane seç
    food = random.sample(food_list, min(len(food_list), 5))

    # Accuracy hesapla
    total_matches += compute_accuracy(person["Recommended Foods"], food)

print(f"Total Matches: {total_matches}")


Processing dataset: 100%|██████████| 495/495 [18:52<00:00,  2.29s/it]

Total Matches: 216


In [22]:
accuracy = total_matches / len(processed_val_dataset)
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 43.64%


In [16]:
model_outputs = []
model_outputs.append(food)

In [19]:
# Doğruluğu hesaplamak için bir fonksiyon tanımlayın
def compute_accuracy(recommended, predicted):
    correct_predictions = len(set(recommended).intersection(set(predicted)))
    return correct_predictions / len(recommended)

accuracy = 0

# Model çıktıları ve gerçek değerleri kıyasla
for person, predicted_food in zip(processed_val_dataset, model_outputs):
    recommended_food = person['recommended foods']  # Gerçek değerler
    accuracy += compute_accuracy(recommended_food, predicted_food)

# Ortalama doğruluğu hesapla
accuracy /= len(processed_val_dataset)
print(f"Model Accuracy: {accuracy:.2f}")


KeyError: 'recommended foods'